In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv, html
import lxml

# Regex
regex_html = r"(<[^>]+>)"
regex_html_4_content = r"(\<\/?[a-zA-Z0-9\=\"\_\s\-]+\>)"
regex_d_space = r"(\s{2,})"
regex_weblink = r"(www.[A-Za-z\.]+)"

# File url
file_url = './training/jobs_May-16-2019.csv'
dest_url = './training/training_May-16-2019.csv'


# Basic list
csv_header = ['title','location','type_empl','wage','industries','content']

# Support scripts
# %run -i scripts.py

In [2]:
# Merge list of string into a long text.
def join_txt(list_txt):
    # Join list of content into 1 simple paragraph
    seperator = '. '
    return seperator.join(list_txt)

# Clear any html tags and decode the text
def clear_html_and_unescape(sample):
    # Clear html
    result = re.sub(regex_html, '', sample[0])
    
    # Clear Unescape
    result = html.unescape(result)
    
    return result

# Clean Web content
def clean_web_scrap_content(content):
    
    content = join_txt(content)
    content = re.sub(r"(<[^>]+>)",' ',content)
    content = html.unescape(content)
    content = re.sub(r"(\s{2,})", ' ',content)
    content = re.sub(r"(<[^>]+>)",'website',content)
    content = content.replace(u'\xa0', u' ').lstrip().rstrip()
    content = re.sub(r'([\•\s]{2,})',' ',content)
    
    return content

In [3]:
frame = pd.read_csv(file_url, sep=',')
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5529 entries, 0 to 5528
Data columns (total 1 columns):
url    5529 non-null object
dtypes: object(1)
memory usage: 43.3+ KB


In [4]:
# -------------------------------------------------
# Grant Url as a list
frame_urls = frame['url'].values

# Attempt title
title_attempt = True
for key, url in enumerate(frame_urls):
    # XPath Strings
    res = requests.get(url)
    if res.status_code  == 200:

        print("Header: ", res.status_code)
        # Initialize XPath
        xpath_selector = Selector(text=res.text)
        
        # Init dictionary
        list_content = dict()
        
        # Print url for tracking
        print(url)

        # Title
        # //h1[@class="jobtitle"]
#         title = xpath_selector.xpath('//h1[@class="jobtitle"]/text()').extract()
        title = xpath_selector.xpath('//title/text()').extract()
        title = title[0]

        # Location
        # //section[@role="region"]/dl/dd[2]/span/span
        location = xpath_selector.xpath('//section[@role="region"]/dl/dd[2]/span/span').extract()
        location = clear_html_and_unescape(location)

        # Type of Employee
        # //section[@role="region"]/dl/dd[3]/span/span
        type_employee = xpath_selector.xpath('//section[@role="region"]/dl/dd[3]/span/span').extract()
        type_employee = re.sub(regex_html,'', type_employee[0])

        # Wage
        # //*[contains(text(),'Salary')]
        is_Salary = xpath_selector.xpath('//*[contains(text(),"Salary")]/text()').extract()
        if is_Salary:
            # TODO
            wage = xpath_selector.xpath('//article/div[1]/div[2]/div/section/dl/div[1]/dd/span/span/text()').extract()
            if wage:
                wage = wage[0]
            else:
                wage = is_Salary
        else:
            wage = ''

        # Industries
        # //section[@role="region"]/dl/div/dd/span/span/strong
        industries = xpath_selector.xpath('//section[@role="region"]/dl/div/dd/span/span/strong').extract()
        industries = clear_html_and_unescape(industries)

        # Content
        # //div[@data-automation="mobileTemplate"]
        content = xpath_selector.xpath('//div[@data-automation="mobileTemplate"]').extract()
        content = clean_web_scrap_content(content)
        
        # Append to dictionary
        list_content.update({
            'title':     title,
            'location':  location,
            'type_empl': type_employee,
            'wage':      wage,
            'industries':industries,
            'content':   content
        })
        
        if title_attempt:
            print("TItle: ", title_attempt)
            df_temp = pd.DataFrame(columns = list(list_content.keys()))
            frame = pd.concat([frame, df_temp])
            print(frame.columns)                
            title_attempt = False

        for k_, v_ in list_content.items():
            print(k_," : " ,len(v_))
            frame.loc[key,k_] = v_
        print("======================\n")
    else:
        print("['ERRORS'] {} , at {}".format(res.status_code, url))

Header:  200
https://www.seek.com.au/job/38867796?type=promoted&searchrequesttoken=923213a8-5d05-4bba-ac2c-0e93b7641580
TItle:  True
Index(['content', 'industries', 'location', 'title', 'type_empl', 'url',
       'wage'],
      dtype='object')
title  :  48
location  :  29
type_empl  :  9
wage  :  0
industries  :  38
content  :  2989



C:\Users\Shaynie\Anaconda3\envs\capstone_app\lib\site-packages\ipykernel_launcher.py:74: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Header:  200
https://www.seek.com.au/job/38999685?type=standout&searchrequesttoken=923213a8-5d05-4bba-ac2c-0e93b7641580
title  :  56
location  :  29
type_empl  :  9
wage  :  0
industries  :  38
content  :  1098

Header:  200
https://www.seek.com.au/job/38986499?type=standout&searchrequesttoken=923213a8-5d05-4bba-ac2c-0e93b7641580
title  :  59
location  :  29
type_empl  :  9
wage  :  0
industries  :  38
content  :  4117

Header:  200
https://www.seek.com.au/job/38985472?type=standout&searchrequesttoken=923213a8-5d05-4bba-ac2c-0e93b7641580
title  :  55
location  :  29
type_empl  :  9
wage  :  0
industries  :  5
content  :  2866

Header:  200
https://www.seek.com.au/job/39008519?type=standout&searchrequesttoken=923213a8-5d05-4bba-ac2c-0e93b7641580
title  :  48
location  :  29
type_empl  :  9
wage  :  0
industries  :  20
content  :  3068

Header:  200
https://www.seek.com.au/job/39003239?type=standard&searchrequesttoken=923213a8-5d05-4bba-ac2c-0e93b7641580
title  :  37
location  :  8
type_

Header:  200
https://www.seek.com.au/job/38998989?type=standard&searchrequesttoken=8302c43d-6174-4d44-8199-3373731a3ecf
title  :  36
location  :  29
type_empl  :  9
wage  :  3
industries  :  38
content  :  1844

Header:  200
https://www.seek.com.au/job/38970272?type=standard&searchrequesttoken=8302c43d-6174-4d44-8199-3373731a3ecf
title  :  51
location  :  8
type_empl  :  9
wage  :  0
industries  :  26
content  :  2135

Header:  200
https://www.seek.com.au/job/38868358?type=standout&searchrequesttoken=8302c43d-6174-4d44-8199-3373731a3ecf
title  :  48
location  :  29
type_empl  :  9
wage  :  0
industries  :  10
content  :  2989

Header:  200
https://www.seek.com.au/job/39005084?type=standard&searchrequesttoken=8302c43d-6174-4d44-8199-3373731a3ecf
title  :  62
location  :  8
type_empl  :  9
wage  :  0
industries  :  38
content  :  3620

Header:  200
https://www.seek.com.au/job/39006544?type=standout&searchrequesttoken=56327096-a9bb-45f1-a2f6-166980b94e62
title  :  37
location  :  8
type_e

Header:  200
https://www.seek.com.au/job/38863666?type=standard&searchrequesttoken=a55b7e44-de99-4dfd-9f44-ad66c9989b31
title  :  44
location  :  29
type_empl  :  9
wage  :  3
industries  :  20
content  :  1957

Header:  200
https://www.seek.com.au/job/38874128?type=standard&searchrequesttoken=a55b7e44-de99-4dfd-9f44-ad66c9989b31
title  :  37
location  :  29
type_empl  :  9
wage  :  0
industries  :  38
content  :  3468

Header:  200
https://www.seek.com.au/job/38862680?type=standard&searchrequesttoken=a55b7e44-de99-4dfd-9f44-ad66c9989b31
title  :  55
location  :  29
type_empl  :  9
wage  :  0
industries  :  26
content  :  3023

Header:  200
https://www.seek.com.au/job/39020030?type=standard&searchrequesttoken=a55b7e44-de99-4dfd-9f44-ad66c9989b31
title  :  44
location  :  26
type_empl  :  9
wage  :  4
industries  :  38
content  :  3566

Header:  200
https://www.seek.com.au/job/38882905?type=standout&searchrequesttoken=a55b7e44-de99-4dfd-9f44-ad66c9989b31
title  :  60
location  :  26
typ

...

Header:  200
https://www.seek.com.au/job/39000084?type=standard&searchrequesttoken=c29a2813-358e-4d25-ab5a-8ef84c783219
title  :  57
location  :  6
type_empl  :  9
wage  :  0
industries  :  38
content  :  2417

Header:  200
https://www.seek.com.au/job/39007255?type=standout&searchrequesttoken=87ed48ce-3acc-4bf9-bfe4-a15554f24010
title  :  89
location  :  41
type_empl  :  9
wage  :  0
industries  :  38
content  :  3550

Header:  200
https://www.seek.com.au/job/38926741?type=standout&searchrequesttoken=87ed48ce-3acc-4bf9-bfe4-a15554f24010
title  :  57
location  :  6
type_empl  :  9
wage  :  50
industries  :  20
content  :  1796

Header:  200
https://www.seek.com.au/job/38955248?type=standard&searchrequesttoken=87ed48ce-3acc-4bf9-bfe4-a15554f24010
title  :  37
location  :  41
type_empl  :  13
wage  :  0
industries  :  38
content  :  3052

Header:  200
https://www.seek.com.au/job/39020456?type=standout&searchrequesttoken=87ed48ce-3acc-4bf9-bfe4-a15554f24010
title  :  91
location  :  38
typ

Header:  200
https://www.seek.com.au/job/38996172?type=standard&searchrequesttoken=12a9240a-f0eb-4f3a-85bb-4e01a857cdcb
title  :  49
location  :  41
type_empl  :  13
wage  :  0
industries  :  20
content  :  2942

Header:  200
https://www.seek.com.au/job/38959219?type=standard&searchrequesttoken=12a9240a-f0eb-4f3a-85bb-4e01a857cdcb
title  :  62
location  :  41
type_empl  :  9
wage  :  16
industries  :  38
content  :  2585

Header:  200
https://www.seek.com.au/job/38980827?type=standout&searchrequesttoken=12a9240a-f0eb-4f3a-85bb-4e01a857cdcb
title  :  85
location  :  41
type_empl  :  9
wage  :  17
industries  :  20
content  :  7314

Header:  200
https://www.seek.com.au/job/38985899?type=standard&searchrequesttoken=3a0ed790-e413-4d38-b1fa-e6801ca3bae1
title  :  76
location  :  41
type_empl  :  9
wage  :  0
industries  :  26
content  :  3811

Header:  200
https://www.seek.com.au/job/38919726?type=standard&searchrequesttoken=3a0ed790-e413-4d38-b1fa-e6801ca3bae1
title  :  48
location  :  41


Header:  200
https://www.seek.com.au/job/38948029?type=standard&searchrequesttoken=afd89e04-6dbf-4e55-aadd-056ea662fa56
title  :  52
location  :  38
type_empl  :  9
wage  :  0
industries  :  20
content  :  1014

Header:  200
https://www.seek.com.au/job/39021674?type=standard&searchrequesttoken=afd89e04-6dbf-4e55-aadd-056ea662fa56
title  :  45
location  :  36
type_empl  :  9
wage  :  0
industries  :  38
content  :  2790

Header:  200
https://www.seek.com.au/job/38914240?type=standard&searchrequesttoken=afd89e04-6dbf-4e55-aadd-056ea662fa56
title  :  60
location  :  41
type_empl  :  9
wage  :  19
industries  :  38
content  :  2538

Header:  200
https://www.seek.com.au/job/38890789?type=standard&searchrequesttoken=c5d51c92-3096-4029-b399-2cc2f064d038
title  :  39
location  :  41
type_empl  :  9
wage  :  0
industries  :  38
content  :  4230

Header:  200
https://www.seek.com.au/job/38960015?type=standout&searchrequesttoken=c5d51c92-3096-4029-b399-2cc2f064d038
title  :  47
location  :  41
ty

Header:  200
https://www.seek.com.au/job/38869098?type=standout&searchrequesttoken=20d87fd5-db1e-4ff6-9549-ab0cad05858b
title  :  56
location  :  41
type_empl  :  9
wage  :  0
industries  :  20
content  :  1357

Header:  200
https://www.seek.com.au/job/38848549?type=standard&searchrequesttoken=20d87fd5-db1e-4ff6-9549-ab0cad05858b
title  :  37
location  :  41
type_empl  :  13
wage  :  0
industries  :  20
content  :  3694

Header:  200
https://www.seek.com.au/job/38921454?type=standard&searchrequesttoken=20d87fd5-db1e-4ff6-9549-ab0cad05858b
title  :  61
location  :  38
type_empl  :  9
wage  :  26
industries  :  20
content  :  2524

Header:  200
https://www.seek.com.au/job/38923756?type=standard&searchrequesttoken=20d87fd5-db1e-4ff6-9549-ab0cad05858b
title  :  50
location  :  41
type_empl  :  9
wage  :  0
industries  :  31
content  :  5025

Header:  200
https://www.seek.com.au/job/38856941?type=standard&searchrequesttoken=20d87fd5-db1e-4ff6-9549-ab0cad05858b
title  :  96
location  :  41
t

Header:  200
https://www.seek.com.au/job/38491237?type=standout&searchrequesttoken=06bf073a-d9bd-4984-adbe-9d4b1b653933
title  :  54
location  :  30
type_empl  :  9
wage  :  0
industries  :  38
content  :  3111

Header:  200
https://www.seek.com.au/job/38992707?type=standard&searchrequesttoken=9b79d5af-ab2c-4eae-920b-6e3f08ca03ec
title  :  66
location  :  42
type_empl  :  9
wage  :  0
industries  :  26
content  :  1875

Header:  200
https://www.seek.com.au/job/38992709?type=standard&searchrequesttoken=9b79d5af-ab2c-4eae-920b-6e3f08ca03ec
title  :  66
location  :  42
type_empl  :  9
wage  :  0
industries  :  11
content  :  1875

Header:  200
https://www.seek.com.au/job/38992708?type=standard&searchrequesttoken=9b79d5af-ab2c-4eae-920b-6e3f08ca03ec
title  :  66
location  :  42
type_empl  :  9
wage  :  0
industries  :  11
content  :  1875

Header:  200
https://www.seek.com.au/job/38995182?type=standard&searchrequesttoken=9b79d5af-ab2c-4eae-920b-6e3f08ca03ec
title  :  66
location  :  42
typ

In [5]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5529 entries, 0 to 5528
Data columns (total 7 columns):
content       5518 non-null object
industries    5518 non-null object
location      5518 non-null object
title         5518 non-null object
type_empl     5518 non-null object
url           5529 non-null object
wage          5518 non-null object
dtypes: object(7)
memory usage: 505.6+ KB


In [6]:
frame.head()

,content,industries,location,title,type_empl,url,wage
0,Queensland Audit Office Investing in analytics...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Analyst - Audit Analytics Job in Brisbane - SEEK,Full Time,https://www.seek.com.au/job/38867796?type=prom...,
1,Our client has a significant track record of d...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Data Scientist - Analytics and AI Job in Brisb...,Full Time,https://www.seek.com.au/job/38999685?type=stan...,
2,Working for this market leader within the Fina...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Head of Data Analytics & Performance Job in Br...,Full Time,https://www.seek.com.au/job/38986499?type=stan...,
3,Full-Time | Permanent | Brisbane CBD About RAC...,Sales,"Brisbane, CBD & Inner Suburbs",Senior Data Analytics Consultant Job in Brisba...,Full Time,https://www.seek.com.au/job/38985472?type=stan...,
4,Queensland Audit Office Investing in analytics...,Science & Technology,"Brisbane, CBD & Inner Suburbs",Analyst - Audit Analytics Job in Brisbane - SEEK,Full Time,https://www.seek.com.au/job/39008519?type=stan...,


In [7]:
frame['content']=frame['content'].map(lambda row: re.sub(r'([\•\s]{2,})',' ',str(row)))

In [13]:
frame[].dropna(label=['content'],axis=1,inplace=True)

TypeError: dropna() got an unexpected keyword argument 'label'

In [8]:
frame['content'][0]

'Queensland Audit Office Investing in analytics, emerging technologies, systems and capabilities Designed workspaces for collaboration and innovation Flexibility in working arrangements About the role QAO uses advanced analytics and emerging technologies to improve the quality of insight. In this role you utilise analytics and emerging technology to improve the efficiency of our audits and innovate and transform how those audits are performed. In the role of Analyst – Audit Analytics AO4, you engage and collaborate with our audit teams and external clients to: deliver analytic solutions that improve the way we deliver audit services using analytics facilitate the interpretation of analytic solutions coach auditors in analytic techniques and tools identify opportunities to automate manual processes source data and apply data governance principles to data received develop and support databases and contribute to audit analytics capability respond to ad hoc analytic requests. Travel throug

In [9]:
frame.to_csv(dest_url,sep='|',index=False)

In [10]:
# Test Cell

url_w_salary = 'https://www.seek.com.au/job/38961481?type=standout&searchrequesttoken=500a5225-80f9-4fcf-88c4-4985e717adf2'
url_w_salary_1 = 'https://www.seek.com.au/job/38988764?type=standout&searchrequesttoken=7dd0818a-ae26-4408-918c-981552979733'
url_wo_salary = 'https://www.seek.com.au/job/38952364?type=standout&searchrequesttoken=bd45b375-6e0c-4572-852c-1bfa7acc2952'

url_generous_salary = 'https://www.seek.com.au/job/38931990?type=standard&searchrequesttoken=ddc9ad23-06e8-42c5-9691-a35a9f7c6c5e'
url_test_random = 'https://www.seek.com.au/job/38843200?type=standard&searchrequesttoken=480f435f-3db5-4612-a554-a31df99c685e'

# XPath Strings
res = requests.get(url_generous_salary)
if res.status_code  == 200:

    print("Header: ", res.status_code)
    # Initialize XPath
    xpath_selector = Selector(text=res.text)
    
    # Title
    # //h1[@class="jobtitle"]
    title = xpath_selector.xpath('//h1[@class="jobtitle"]/text()').extract()

    # Location
    # //section[@role="region"]/dl/dd[2]/span/span
    location = xpath_selector.xpath('//section[@role="region"]/dl/dd[2]/span/span').extract()
    location = clear_html_and_unescape(location)
    
    # Type of Employee
    # //section[@role="region"]/dl/dd[3]/span/span
    type_employee = xpath_selector.xpath('//section[@role="region"]/dl/dd[3]/span/span').extract()
    type_employee = re.sub(regex_html,'', type_employee[0])
    
    # Wage
    # //*[contains(text(),'Salary')]
    is_Salary = xpath_selector.xpath('//*[contains(text(),"Salary")]/text()').extract()
    if is_Salary:
        # TODO
        wage = xpath_selector.xpath('//article/div[1]/div[2]/div/section/dl/div[1]/dd/span/span/text()').extract()
        wage = wage[0]
    else:
        wage = ''
        
    # Industries
    # //section[@role="region"]/dl/div/dd/span/span/strong
    industries = xpath_selector.xpath('//section[@role="region"]/dl/div/dd/span/span/strong').extract()
    industries = clear_html_and_unescape(industries)
    
    # Content
    # //div[@data-automation="mobileTemplate"]
    content = xpath_selector.xpath('//div[@data-automation="mobileTemplate"]').extract()
    content = clean_web_scrap_content(content)
    
    # Print-out
    print("Title: ", title[0])
    print("Location: ", location)
    print("Type Empl: ", type_employee)
    print("Wage: ", wage)
    print("Industries: ", industries)
    print("Content: ", len(content))
else:
    print("[ERROR]: ", res.status_code)

Header:  200
Title:  Business Development Manager | High Profile Investment Research Firm 
Location:  Sydney, CBD, Inner West & Eastern Suburbs
Type Empl:  Full Time
Wage:  Generous salary!
Industries:  Banking & Financial Services
Content:  2407


In [11]:
content

'Our client has a proud record of analysis and advice, backed by depth of data and research sources. Leading the way in innovation, providing tailored solutions for all investment, strategic and operational goals. Our client is looking for a Business Development Manager to manage the account strategy and sales process for clients. In this role, you will be responsible for relationship management and new business development across all facets of entities. Responsibilities of the role include but are not limited to: Gather company information to identify strategic accounts, generate interest and determine potential Manage sales processes including closing of sales Respond to Requests for Information and Proposals Assist with creation and execution of marketing events and campaigns Coordinate resources to meet sales targets Develop knowledge of the client’s product offerings Build and maintain relationships with clients Engage with clients and provide awareness of the client’s ongoing bus